In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.applications.inception_v3 import decode_predictions
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import keras

Using TensorFlow backend.


In [2]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation, Dropout, Conv2D

In [3]:
num_classes = 7 #angry, disgust, fear, happy, sad, surprise, neutral
batch_size = 20
epochs = 20

In [4]:
with open('data/fer2013.csv') as f:
    content = f.readlines()
lines = np.array(content)

In [5]:
num_of_instances = lines.size
print("num_of_instance", num_of_instances)
print("instance length: ",len(lines[1].split(",")[1].split(" ")))

num_of_instance 35888
instance length:  2304


In [6]:
x_train, y_train, x_test, y_test = [], [], [], []
for i in range(1, num_of_instances):
    try:
        emotion, img, usage = lines[i].split(",")
        val = img.split(" ")
        pixels = np.array(val,'float32')
        emotion = keras.utils.to_categorical(emotion, num_classes)
        
        if "Training" in usage:
            x_train.append(pixels)
            y_train.append(emotion)
        elif "PublicTest" in usage:
            x_test.append(pixels)
            y_test.append(emotion)
    except Exception as e:
        print(e,end="/n")
x_train = np.array(x_train, 'float32')
y_train = np.array(y_train, 'float32')
x_test = np.array(x_test, 'float32')
y_test = np.array(y_test, 'float32')

x_train /= 255 #normalize inputs between [0, 1]
x_test /= 255

x_train = x_train.reshape(x_train.shape[0], 48, 48, 1)
x_train = x_train.astype('float32')
x_test = x_test.reshape(x_test.shape[0], 48, 48, 1)
x_test = x_test.astype('float32')

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

28709 train samples
3589 test samples


In [7]:
conv_base = VGG16(include_top=False, weights='imagenet', input_shape=(48, 48, 3))
model = Sequential()
model.add(Conv2D(3, kernel_size=(1,1), input_shape=(48,48,1), activation='relu'))
model.add(conv_base)
model.add(Flatten())
model.add(Dense(1024, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))
conv_base.trainable = False

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 3)         6         
_________________________________________________________________
vgg16 (Model)                (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 7175      
Total params: 15,247,181
Trainable params: 532,493
Non-trainable params: 14,714,688
__________________________________________________________

In [ ]:
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
generator = image.ImageDataGenerator()
train_gen = generator.flow(x_train, y_train, batch_size=batch_size)
generator = image.ImageDataGenerator()
valid_gen = generator.flow(x_test, y_test, batch_size=batch_size)

In [ ]:
history = model.fit_generator(
                            train_gen,
                            steps_per_epoch=len(x_train)//batch_size,
                            epochs=20,
                            validation_data=valid_gen,
                            validation_steps=179)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
1435/1435 [==============================] - 1823s 1s/step - loss: 1.6925 - acc: 0.3257 - val_loss: 1.6143 - val_acc: 0.3645
Epoch 2/20
 482/1435 [=========>....................] - ETA: 19:18 - loss: 1.6182 - acc: 0.3595